# cleaning des valeur fonciere;
- recup du type de bien (maison et appartement), valeur et code INSSE.
- groupage par code INSEE et moyenne des valeurs.

In [8]:
import pandas as pd
import seaborn as sns
import os #import de l'Os
from dotenv import load_dotenv, find_dotenv
import ssl
import platform
a = platform.system()

ssl._create_default_https_context = ssl._create_unverified_context

if a == 'Windows':
    load_dotenv("./.env",  override=True)
else:
    load_dotenv( dotenv_path="./.env.local", override=True)
cleandata = os.environ.get("EXPORTDIR")

In [6]:
# WARNING ! 500mo de téléchargement
df_VF = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/78348f03-a11c-4a6b-b8db-2acf4fee81b1", sep='|', decimal=',', dtype={"Code postal": "str"} )

C:\Users\thierry\AppData\Local\Temp\ipykernel_12292\2346662484.py:1: DtypeWarning: Columns (14,18,23,24,26,28,32,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_VF = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/78348f03-a11c-4a6b-b8db-2acf4fee81b1", sep='|', decimal=',', dtype={"Code postal": "str"} )


In [10]:
df_VF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 43 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Identifiant de document     float64
 1   Reference document          float64
 2   1 Articles CGI              float64
 3   2 Articles CGI              float64
 4   3 Articles CGI              float64
 5   4 Articles CGI              float64
 6   5 Articles CGI              float64
 7   No disposition              int64  
 8   Date mutation               object 
 9   Nature mutation             object 
 10  Valeur fonciere             float64
 11  No voie                     float64
 12  B/T/Q                       object 
 13  Type de voie                object 
 14  Code voie                   object 
 15  Voie                        object 
 16  Code postal                 object 
 17  Commune                     object 
 18  Code departement            object 
 19  Code commune         

In [11]:
df_VF.columns

Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')

In [12]:
# uniquement les colonnes necessaire et filtrage sur maison / appartement
df_VF = df_VF[["Type local", "Valeur fonciere", 'Code departement', 'Code commune']]

In [13]:
df_VF = df_VF[(df_VF['Type local'] == 'Appartement') | (df_VF['Type local'] == 'Maison')]

df_VF.head()

,Type local,Valeur fonciere,Code departement,Code commune
2,Appartement,1070000.0,1,354
3,Maison,152200.0,1,404
4,Maison,269000.0,1,361
7,Maison,770000.0,1,281
14,Maison,85520.0,1,24


In [14]:
#code insee
df_VF["Code departement"].astype("str")
df_VF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1082996 entries, 2 to 3727004
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Type local        1082996 non-null  object 
 1   Valeur fonciere   1078066 non-null  float64
 2   Code departement  1082996 non-null  object 
 3   Code commune      1082996 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 41.3+ MB


In [15]:
def add_zero(x):
    if len(str(x)) < 2:
        return '0' + str(x)
    else:
        return x

In [16]:
def add_2zero(x):
    if len(str(x)) == 2:
        return '0' + str(x)
    if len(str(x)) == 1:
        return '00' + str(x)
    else:
        return x

In [17]:
df_VF['Code departement'] = df_VF['Code departement'].apply(add_zero)

In [18]:
df_VF['Code commune'] = df_VF['Code commune'].apply(add_2zero)
df_VF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1082996 entries, 2 to 3727004
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Type local        1082996 non-null  object 
 1   Valeur fonciere   1078066 non-null  float64
 2   Code departement  1082996 non-null  object 
 3   Code commune      1082996 non-null  object 
dtypes: float64(1), object(3)
memory usage: 41.3+ MB


In [19]:
df_VF['Code INSEE'] = df_VF['Code departement'].astype('str') + df_VF['Code commune'].astype('str')

In [20]:
df_VF = df_VF.drop(['Code departement', 'Code commune'], axis=1)

In [21]:
df_VF.dropna()
df_VF.describe()

,Valeur fonciere
count,1.078066e+06
mean,6.016794e+05
std,4.828804e+06
min,1.500000e-01
25%,1.200000e+05
50%,2.000000e+05
75%,3.309205e+05
max,7.620000e+08


In [22]:
df_VF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1082996 entries, 2 to 3727004
Data columns (total 3 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Type local       1082996 non-null  object 
 1   Valeur fonciere  1078066 non-null  float64
 2   Code INSEE       1082996 non-null  object 
dtypes: float64(1), object(2)
memory usage: 33.1+ MB


In [23]:
df_VF_commune= df_VF.groupby(['Code INSEE', 'Type local']).mean()

In [24]:
df_VF_commune.head()

Valeur fonciere
Code INSEE Type local                  
01001      Maison         319333.333333
01002      Appartement     90000.000000
           Maison         162900.000000
01004      Appartement    179329.354839
           Maison         256139.539216

In [25]:
PathClean = os.environ.get("EXPORTDIR")
df_VF_commune.to_csv(os.path.join(PathClean, "valeursfoncieres-2023.csv"))